# Population Statistics

In [11]:
### Imports

In [34]:
%pip install wbdata
%pip install pandas
%pip install iso3166

import wbdata
import pandas
import math
import iso3166

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
## Topic Selection

In [25]:
# Returns the number of displaced people given the country/region
# This allows us to access any listed country/region within wbdata and see the number of people displaced between certain years
# **IMPORTANT: Some countries didn't track certain years, so a timeframe that matches most countries is required (eg. 2008-2017)**

population_displaced = wbdata.get_indicator(source=2) # World Development Indicators
variable = {"VC.IDP.NWDS": "Internally displaced persons, new displacement associated with disasters (number of cases)"} #SOURCE

dataframe = wbdata.get_dataframe(variable) # Creates dataframe
cleaned_df = dataframe.dropna() # Drops NaN values

df_index = cleaned_df.reset_index() # DF is in multi-indexed format; this resets the index to be more easily accessed
filtered_df = df_index[df_index['date'] == "2022"] # Select location
 
filtered_df.sort_values(by='Internally displaced persons, new displacement associated with disasters (number of cases)', ascending=False).head(5)

,country,date,"Internally displaced persons, new displacement associated with disasters (number of cases)"
541,World,2022,32533018.0
320,Low & middle income,2022,31585676.0
186,IDA & IBRD total,2022,31511406.0
383,Middle income,2022,26735476.0
348,Lower middle income,2022,21218356.0


In [29]:
# Returns the population size of a given country
# SOURCE = 40; Population estimates and projections

world_population = {"SP.POP.TOTL":"World Population"}
female_population = {"SP.POP.TOTL.FE.IN": "Population, female"}
male_population = {"SP.POP.TOTL.MA.IN":"Population, male"}

total_count = wbdata.get_dataframe(world_population, country="WLD") #"WLD" will be replaced with the given country ID as a parameter
male_count = wbdata.get_dataframe(male_population, country="WLD")
female_count = wbdata.get_dataframe(female_population, country="WLD")

new = total_count.reset_index()
filtered = new[new['date'] == '2022']

print(filtered)



   date  World Population
0  2022      7.950947e+09


In [53]:
def population(year, sex, age_range, location):
    
    # Changes location into valid alpha3 country code
    alpha3_name = iso3166.countries.get(location).alpha3
    
    def age_helper(age_range):

        range = []

        # Rounds age up/down to fit within the age range of datasets
        lower_bound = 5 * math.floor(min(age_range)/5)
        upper_bound = 5 * round(max(age_range)/5)
        
        # Loops for every dataframe that contains lower_bound and lower_bound + 4 (example: {"6064"})
        while lower_bound < min(79, upper_bound + 1):
            range.append("{:02}{:02}".format(lower_bound, lower_bound + 4))
            lower_bound += 5

        # Adds the ID "80UP" to range when the upper_bound is greater than 80
        if upper_bound >= 80:
            range.append("80UP")

        return range
        
    # Filters sex parameter into a specific ID to access
    if sex == "male" or "Male":
        sex = "MA"
    elif sex == "female" or "Female":
        sex = "FE"
    else:
        print("Please enter 'Male' or 'Female'")
        
    newLst = []
    for i in age_helper(age_range):
        df_input = "SP.POP.{}.{}".format(i, sex)
        newLst.append(df_input)
        
    final = "this is the year: {}, the sex: {}, wbdata frame: {}, and alpha3 name: {}".format(year, sex, newLst, alpha3_name)
    print(final)

population(1966, "Male", (18,426), "USA")

this is the year: 1966, the sex: MA, wbdata frame: ['SP.POP.1519.MA', 'SP.POP.2024.MA', 'SP.POP.2529.MA', 'SP.POP.3034.MA', 'SP.POP.3539.MA', 'SP.POP.4044.MA', 'SP.POP.4549.MA', 'SP.POP.5054.MA', 'SP.POP.5559.MA', 'SP.POP.6064.MA', 'SP.POP.6569.MA', 'SP.POP.7074.MA', 'SP.POP.7579.MA', 'SP.POP.80UP.MA'], and alpha3 name: USA
